## Part0 輸入照片

![](./ShadowStrip/0030.jpg)

## Part1 二值化原始輸入照片

In [1]:
import cv2
import numpy as np
import os

# 檔案路徑設定
#=========================================================

# 輸入
input_directory = "ShadowStrip"
output_directory = "RGB_threshold_RedBinary"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

#==========================================================

# 定義紅色的 RGB 範圍
red_threshold = 90  # 調整此值以適應不同圖像

# 遍歷資料夾中的所有圖像檔案
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):
        # 讀取圖像
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        # 提取圖像的 RGB 通道
        r_channel = image[:, :, 2]  # 紅色通道
        g_channel = image[:, :, 1]  # 綠色通道
        b_channel = image[:, :, 0]  # 藍色通道

        # 判斷紅色通道是否遠高於綠色和藍色通道
        red_mask = (r_channel > red_threshold) & (r_channel > g_channel+10) & (r_channel > b_channel+10)

        # 將結果轉換為二值化圖像
        red_binary = np.zeros_like(r_channel, dtype=np.uint8)
        red_binary[red_mask] = 255

        # 儲存結果
        output_filename = os.path.join(output_directory, filename.replace('.jpg', '_RGB_threshold_RedBinary.jpg'))
        cv2.imwrite(output_filename, red_binary)
        # print(f"Binary image of red regions saved as {output_filename}")

print("All images processed Mask result successfully.")
print(f"Binary image of red regions saved at {output_directory}")



All images processed Mask result successfully.
Binary image of red regions saved at RGB_threshold_RedBinary


![](./RGB_threshold_RedBinary/0030_RGB_threshold_RedBinary.jpg)

## Part2 找出框上面小紅色區段的平均座標點 並且執行產生兩方向的homography結果

##### 1. (非必要)固定標記好方框端點 先劃出一下方框與直線(可視易閱讀端點座標與順序編號) 有助於後續計算水平4個梁的直線方程式係數

In [2]:
'''畫方框圖只是示意 但是還是做一下 比較心安 現在不需要了 其實下面已經有寫純用8點計算直線參數了'''
import cv2
import numpy as np
import os

# 檔案路徑設定
#=========================================================

# 輸入
input_directory = "ShadowStrip"
input_file_name = "0000.jpg"

# 輸出
output_directory = "box_frame_line"
output_file_name = "box_frame_line.jpg"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

#==========================================================

# 讀取圖像
image_path = os.path.join(input_directory, input_file_name)
image = cv2.imread(image_path)

# 定義八個座標點
points = [(238, 67), (988, 123), (270, 998), (959, 849),
          (72,157), (619,181), (103,760), (617, 690)]

# 在圖像上畫四條線段
for i in range(4):
    cv2.line(image, points[i*2], points[(i*2)+1], (0, 0, 255), 2)
# 在點上標上數字
for i in range(8):
    cv2.putText(image, str((i+1)), (points[i][0] - 30, points[i][1] - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.circle(image, points[i], 10, (0, 255, 0), -1)  # 在座標點位置繪製綠色實心圓

# 計算直線方程式的係數
for i in range(4):
    x1, y1 = points[i*2]
    x2, y2 = points[(i*2)+1]
    vx, vy, x0, y0 = cv2.fitLine(np.array([[x1, y1], [x2, y2]], dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)
    slope = vy / vx
    intercept = y0 - (slope * x0)
    print(f"Line {i+1} equation: y = {slope}x + {intercept}")
    print(" vx = ",vx,"vy = ",vy,"x0 = ",x0,"y0 = ",y0)

# 儲存結果
output_filename = os.path.join(output_directory, input_file_name.replace('.jpg', '_' + output_file_name))
cv2.imwrite(output_filename, image)

print("All images processed Mask result successfully.")
print(f"Binary image of red regions saved at {output_directory}")


Line 1 equation: y = [0.07466667]x + [49.229332]
 vx =  [0.99722403] vy =  [0.0744594] x0 =  [613.] y0 =  [95.]
Line 2 equation: y = [-0.21625544]x + [1056.3889]
 vx =  [0.97740626] vy =  [-0.21136943] x0 =  [614.5] y0 =  [923.5]
Line 3 equation: y = [0.04387569]x + [153.84094]
 vx =  [0.9990389] vy =  [0.04383352] x0 =  [345.5] y0 =  [169.]
Line 4 equation: y = [-0.13618676]x + [774.0272]
 vx =  [0.9908536] vy =  [-0.13494115] x0 =  [360.] y0 =  [725.]
All images processed Mask result successfully.
Binary image of red regions saved at box_frame_line


![](box_frame_line\0000_box_frame_line.jpg)

#### 2. 產生沒有把所有雕像都block掉 output_directory = "statue_blocked"

In [3]:
'''遍歷版本 把所有雕像都block掉'''
import cv2
import os

# 檔案路徑設定
#=========================================================

# 輸入目錄
input_directory = "RGB_threshold_RedBinary"

# 輸出目錄
output_directory = "statue_blocked"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

#==========================================================

# 遍歷輸入目錄中的所有檔案
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):  # 確保只處理 .jpg 檔案
        # 讀取圖像
        input_file_path = os.path.join(input_directory, filename)
        image = cv2.imread(input_file_path)

        # 定義方框範圍的最左上與最右下座標
        top_left = (365, 235)
        bottom_right = (600, 680)

        # 將指定範圍內的像素值設為全黑
        image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]] = 0

        # 儲存結果
        output_filename = os.path.join(output_directory, filename.replace('.jpg', '_statue_blocked.jpg'))
        cv2.imwrite(output_filename, image)

        # print(f"Processed image saved at {output_filename}")


![](statue_blocked\0030_RGB_threshold_RedBinary_statue_blocked.jpg)

#### 3.計算求得 homography scr_points 每張圖都有四個點 (在圖像上畫出平均座標點並標上群組編號)

In [4]:
"""平均座標點旁邊都會標上相應的群組編號。 遍歷版本 """

import cv2
import numpy as np
import os

def distance_to_line(x_prime, y_prime, line_params):
    """
    計算其他座標點到直線的距離。

    參數:
    - x_prime: 其他座標點的 x 座標。
    - y_prime: 其他座標點的 y 座標。
    - line_params: 包含直線參數的向量 (vx, vy, x0, y0)。

    返回:
    - 該座標點到直線的距離。
    """
    vx, vy, x0, y0 = line_params
    distance = np.abs((x_prime - x0) * vy - (y_prime - y0) * vx) / np.sqrt(vx**2 + vy**2)
    return distance

# 定義八個座標點
points = [(238, 67), (988, 123), (270, 998), (959, 849),
          (72,157), (619,181), (103,760), (617, 690)]

# 存儲直線方程的係數向量
line_params_list = []

# !!!!!!!!!儲存  遍歷所有照片結果 平均四點座標 的大矩陣!!!!!!!!!!!!!!!
avg_coordinates_matrix = []

# 計算直線方程式的係數
for i in range(4):
    x1, y1 = points[i*2]
    x2, y2 = points[(i*2)+1]
    vx, vy, x0, y0 = cv2.fitLine(np.array([[x1, y1], [x2, y2]], dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)
    line_params_list.append((vx, vy, x0, y0))

# 建立輸入目錄
input_directory = "statue_blocked"

# 建立輸出目錄
output_directory = "box_frame_4_points"
output_file_name = "box_frame_4_points.jpg"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 遍歷目錄中的所有.jpg檔案
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):
        # 讀取圖像
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        # 找出所有白色點的座標
        white_points = np.where(image == 255)
        white_points = np.column_stack((white_points[1], white_points[0]))  # 將座標點轉換成 (x, y) 格式

        # 初始化四個群組
        grouped_coordinates = [[] for _ in range(4)]

        # 將白色點分組
        for point in white_points:
            distances = [distance_to_line(point[0], point[1], line_params) for line_params in line_params_list]
            min_distance_index = np.argmin(distances)
            grouped_coordinates[min_distance_index].append(point)

        # 計算每個群組的平均值
        avg_coordinates = [(np.mean(group, axis=0)[0], np.mean(group, axis=0)[1]) for group in grouped_coordinates]

        # 儲存平均四點座標到矩陣中
        # !!!!!!!!!儲存  遍歷所有照片結果 平均四點座標 的大矩陣!!!!!!!!!!!!!!!
        avg_coordinates_matrix.append(avg_coordinates)

# ================以下只是畫圖======儲存標記四個點=================
        # 在圖像上畫出平均座標點並標上群組編號
        cross_size = (50, 50)  # 十字的大小，(橫向大小, 縱向大小)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_thickness = 1
        for i, coord in enumerate(avg_coordinates, 1):
            x, y = int(coord[0]), int(coord[1])
            cross_half_size = (cross_size[0] // 2, cross_size[1] // 2)
            cv2.line(image, (x - cross_half_size[0], y), (x + cross_half_size[0], y), (0, 255, 0), 1)  # 橫向線
            cv2.line(image, (x, y - cross_half_size[1]), (x, y + cross_half_size[1]), (0, 255, 0), 1)  # 縱向線
            cv2.putText(image, f"Group {i}", (x + 15, y - 15), font, font_scale, (0, 255, 0), font_thickness, cv2.LINE_AA)

        # 儲存結果
        output_filename = os.path.join(output_directory, filename.replace('.jpg', '_' + output_file_name))
        cv2.imwrite(output_filename, image)

print(avg_coordinates_matrix)

[[(558.6603773584906, 90.50943396226415), (560.2608695652174, 935.0652173913044), (295.2352941176471, 166.64705882352942), (310.9230769230769, 730.5384615384615)], [(558.7777777777778, 91.17777777777778), (560.725, 934.625), (295.5, 166.5), (311.7692307692308, 730.3846153846154)], [(559.6451612903226, 91.33870967741936), (560.936170212766, 934.0212765957447), (296.42857142857144, 166.64285714285714), (311.75, 730.6875)], [(560.875, 91.203125), (562.0416666666666, 934.1041666666666), (296.75, 167.16666666666666), (312.6923076923077, 730.0769230769231)], [(561.8039215686274, 90.7843137254902), (563.6382978723404, 933.4468085106383), (297.3125, 166.9375), (313.6470588235294, 730.2941176470588)], [(563.8809523809524, 91.88095238095238), (565.3913043478261, 934.1086956521739), (299.1111111111111, 166.77777777777777), (315.3333333333333, 729.9166666666666)], [(565.9803921568628, 91.2156862745098), (567.0, 933.1304347826087), (300.93333333333334, 166.86666666666667), (316.84615384615387, 729.

![](box_frame_4_points\0030_RGB_threshold_RedBinary_statue_blocked_box_frame_4_points.jpg)

#### 4. 根據求得座標點 homography 到方框 1.正面 2.側面

In [5]:
# 定義八個座標點 以及印出 遍歷所有照片結果 平均四點座標 的大矩陣!!(55, 4, 2)
points = [(238, 67), (988, 123), (270, 998), (959, 849),
          (72,157), (619,181), (103,760), (617, 690)]

"""!!!!!!!!!儲存  遍歷所有照片結果 平均四點座標 的大矩陣!!(55, 4, 2)"""
# 列出欄位資料型別等資訊 
np.shape(avg_coordinates_matrix) # (55, 4, 2)


(55, 4, 2)

#### 4.1 homography至 1.側面

In [6]:
import cv2
import numpy as np
import os

# 讀取所有.jpg文件
input_directory = "RGB_threshold_RedBinary"
output_directory = "left_view_slice"


# 檢查輸出目錄是否存在，若不存在則建立
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 初始化 avg_coordinates_matrix
# 假設 avg_coordinates_matrix 是一個大小為 (55, 4, 2) 的 NumPy 矩陣
# 55 張照片，每張照片有 4 個點，每個點有 x 和 y 座標
# 在這裡，我們假設 avg_coordinates_matrix 已經被定義和填充
print("avg_coordinates_matrix: ", np.shape(avg_coordinates_matrix))

# 遍歷所有.jpg文件
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):
        # 載入原始圖像
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        # 取出對應的平均座標點
        index = int(filename.split("_")[0])  # 假設文件名是"index_...jpg"
        avg_coordinates = avg_coordinates_matrix[index]
        avg_coordinates = np.array(avg_coordinates, dtype=np.float32)

        # 定義目標四個角的坐標（從右上、右下、左上、左下的順序）
        dst_points = np.array([[1080, 0],
                               [1080, 1080],
                               [0, 0],
                               [0, 1080]], dtype=np.float32)

        # 執行透視變換（homography）
        H, _ = cv2.findHomography(avg_coordinates, dst_points)

        # 對原始圖像應用透視變換
        warped_image = cv2.warpPerspective(image, H, (1080, 1080))

        # 儲存結果
        output_filename = os.path.join(output_directory, filename.replace('.jpg', '_left_view_slice.jpg'))
        cv2.imwrite(output_filename, warped_image)

print("Left view slice Homography transformation completed and saved successfully.")


avg_coordinates_matrix:  (55, 4, 2)
Left view slice Homography transformation completed and saved successfully.


![](left_view_slice\0030_RGB_threshold_RedBinary_left_view_slice.jpg)

#### 4.2 homography至 2.正面

In [7]:
import cv2
import numpy as np
import os

# 定義四個點的坐標（左上、右上、左下、右下的順序）
src_points = np.array([[238, 67],
                       [988, 123],
                       [270, 998],
                       [959, 849]], dtype=np.float32)

# 讀取所有.jpg文件
input_directory = "RGB_threshold_RedBinary"
output_directory = "front_view_slice"


# 檢查輸出目錄是否存在，若不存在則建立
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 遍歷所有.jpg文件
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):
        # 載入原始圖像
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        # 定義目標四個角的坐標（左上、右上、左下、右下的順序）
        dst_points = np.array([[0, 0],
                               [1080, 0],
                               [0, 1080],
                               [1080, 1080]], dtype=np.float32)

        # 執行透視變換（homography）
        H, _ = cv2.findHomography(src_points, dst_points)

        # 對原始圖像應用透視變換
        warped_image = cv2.warpPerspective(image, H, (1080, 1080))

        # 儲存結果
        output_filename = os.path.join(output_directory, filename.replace('.jpg', '_front_view_slice.jpg'))
        cv2.imwrite(output_filename, warped_image)

print("Front view slice Homography transformation completed and saved successfully.")


Front view slice Homography transformation completed and saved successfully.


![](front_view_slice\0030_RGB_threshold_RedBinary_front_view_slice.jpg)

## Part3 計算座標&轉換至實際尺寸&合併輸出成點雲.xyz

#### 1.側面 生成 xy座標.txt 每一張的(x,y) 儲存至 ./X_Y_coordinates_output/ 

In [8]:
import cv2
import numpy as np
import os

def convert_pixel_to_mm(pixel_value, image_width, mm_range):
    """
    將像素值轉換為毫米值。

    參數：
    - pixel_value：要轉換的像素值。
    - image_width：圖像的寬度（單位：像素）。
    - mm_range：圖像寬度對應到的毫米範圍。

    返回：
    - 轉換後的毫米值。
    """
    mm_per_pixel = mm_range / (image_width - 1)  # 確保最右邊像素也包含在範圍內
    mm_value = ((pixel_value - (image_width - 1) / 2) * mm_per_pixel)  # 將最左邊的像素設為 -98.5mm
    return mm_value

def process_image(input_directory, output_directory):
    # 遍歷資料夾中的所有圖像檔案
    for filename in os.listdir(input_directory):
        if filename.endswith(".jpg"):
            # 讀取圖像
            input_file_path = os.path.join(input_directory, filename)
            image = cv2.imread(input_file_path)

            # 定義方框範圍的最左上與最右下座標
            top_left = (50, 50)
            bottom_right = (1030, 1030)

            # 將指定範圍外的像素值設為全黑
            mask = np.zeros_like(image)
            cv2.rectangle(mask, top_left, bottom_right, (255, 255, 255), -1)
            image[mask == 0] = 0

            # 找出範圍內的白色像素的座標
            white_points = np.where(image == 255)
            coordinates = list(zip(white_points[1], white_points[0]))  # 將(x, y)座標轉換為(y, x)

            # 計算座標的毫米值
            mm_range = 2 * 98.5  # 毫米範圍
            mm_coordinates = [(convert_pixel_to_mm(x, 1080, mm_range), convert_pixel_to_mm(y, 1080, mm_range)) for x, y in coordinates]

            # 將座標儲存為.txt文件
            output_filename = os.path.join(output_directory, filename.replace('.jpg', '_coordinates.txt'))
            with open(output_filename, 'w') as f:
                for coord in mm_coordinates:
                    f.write(f"{coord[0]:.2f} {coord[1]:.2f}\n")

            # print(f"Converted coordinates saved at {output_filename}")

            # 返回白點數量 確認用而已
            # white_pixel_count = len(coordinates)
            # print(f"Number of white pixels in {filename}: {white_pixel_count}")

# 檔案路徑設定
input_directory = "left_view_slice"
output_directory = "X_Y_coordinates_output"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 呼叫處理函數
process_image(input_directory, output_directory)
print(f" All X_Y_coordinate saved at {output_directory}")


 All X_Y_coordinate saved at X_Y_coordinates_output


[0030_RGB_threshold_RedBinary_left_view_slice_coordinates.txt](X_Y_coordinates_output/0030_RGB_threshold_RedBinary_left_view_slice_coordinates.txt)

#### 2. 正面 生成 z座標.txt 每一張的(x,y) 儲存至 ./Z_coordinates_output/ 

#### 2.1 (非必要)生成可視化的Z資訊圖片 (在正面結果上面劃上平均垂直線 作為Z 並標上實際尺寸mm)

In [9]:
"""下面只有劃線儲存照片的 暫時不需要了 因為我還要存Z"""
import cv2
import numpy as np
import os

def convert_pixel_to_mm(pixel_value, image_width, mm_range):
    """
    將像素值轉換為毫米值。

    參數：
    - pixel_value：要轉換的像素值。
    - image_width：圖像的寬度（單位：像素）。
    - mm_range：圖像寬度對應到的毫米範圍。

    返回：
    - 轉換後的毫米值。
    """
    mm_per_pixel = mm_range / (image_width - 1)  # 確保最右邊像素也包含在範圍內
    mm_value = ((pixel_value - (image_width - 1) / 2) * mm_per_pixel)  # 將最左邊的像素設為 -98.5mm
    return mm_value

# 檔案路徑設定
#=========================================================

# 輸入
input_directory = "front_view_slice"
# 輸出
output_directory = "Z_value_line"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

#==========================================================


# 遍歷資料夾中的所有圖像檔案
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):
        # 讀取圖像
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        # 定義方框範圍的最左上與最右下座標
        top_left = (0, 50)
        bottom_right = (1080, 1030)

        # 將指定範圍內的像素值設為全黑
        image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]] = 0

        # 找出剩餘白點的 x 座標
        white_points = np.where(image == 255)
        remaining_x_coordinates = white_points[1]

        # 計算 x 座標的平均值
        x_avg = int(np.mean(remaining_x_coordinates))

        # 繪製一條綠色的垂直線
        cv2.line(image, (x_avg, 0), (x_avg, 1080), (0, 255, 0), 2)

        # 在線旁邊標記 X 值
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_color = (255, 255, 255)
        cv2.putText(image, f"X = {x_avg}", (x_avg + 10, 50), font, font_scale, font_color, 2)

        # 計算毫米值
        mm_range = 2 * 98.5  # 毫米範圍
        mm_value = convert_pixel_to_mm(x_avg, 1080, mm_range)

        # 在線旁邊標記毫米值
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_color = (255, 255, 255)
        cv2.putText(image, f"{mm_value:.2f} mm", (x_avg + 10, 100), font, font_scale, font_color, 2)


        # 儲存結果
        output_filename = os.path.join(output_directory, filename.replace('.jpg', 'Z_value_line.jpg'))
        cv2.imwrite(output_filename, image)

print(f" All Z_value_line image of red regions saved at {output_directory}")

 All Z_value_line image of red regions saved at Z_value_line


![](Z_value_line\0030_RGB_threshold_RedBinary_front_view_sliceZ_value_line.jpg)

#### 2.2儲存Z座標.txt至./Z_coordinates_output/ 

In [10]:
import cv2
import numpy as np
import os

def convert_pixel_to_mm(pixel_value, image_width, mm_range):
    """
    將像素值轉換為毫米值。

    參數：
    - pixel_value：要轉換的像素值。
    - image_width：圖像的寬度（單位：像素）。
    - mm_range：圖像寬度對應到的毫米範圍。

    返回：
    - 轉換後的毫米值。
    """
    mm_per_pixel = mm_range / (image_width - 1)  # 確保最右邊像素也包含在範圍內
    mm_value = ((pixel_value - (image_width - 1) / 2) * mm_per_pixel)  # 將最左邊的像素設為 -98.5mm
    return mm_value

# 檔案路徑設定
#=========================================================

# 輸入
input_directory = "front_view_slice"
# 輸出
output_image_directory = "Z_value_line"
output_txt_directory = "Z_coordinates_output"

if not os.path.exists(output_image_directory):
    os.makedirs(output_image_directory)

if not os.path.exists(output_txt_directory):
    os.makedirs(output_txt_directory)

# 儲存 mm_value 值的列表
mm_value_list = []

#==========================================================


# 遍歷資料夾中的所有圖像檔案
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):
        # 讀取圖像
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        # 定義方框範圍的最左上與最右下座標
        top_left = (0, 50)
        bottom_right = (1080, 1030)

        # 將指定範圍內的像素值設為全黑
        image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]] = 0

        # 找出剩餘白點的 x 座標
        white_points = np.where(image == 255)
        remaining_x_coordinates = white_points[1]

        # 計算 x 座標的平均值
        x_avg = int(np.mean(remaining_x_coordinates))

        # 計算毫米值
        mm_range = 2 * 98.5  # 毫米範圍
        mm_value = convert_pixel_to_mm(x_avg, 1080, mm_range)
        mm_value_list.append(mm_value)

        # 繪製一條綠色的垂直線
        cv2.line(image, (x_avg, 0), (x_avg, 1080), (0, 255, 0), 2)

        # 在線旁邊標記 X 值
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_color = (255, 255, 255)
        cv2.putText(image, f"X = {x_avg}", (x_avg + 10, 50), font, font_scale, font_color, 2)

        # 在線旁邊標記毫米值
        cv2.putText(image, f"{mm_value:.2f} mm", (x_avg + 10, 100), font, font_scale, font_color, 2)

        # 儲存結果
        output_image_filename = os.path.join(output_image_directory, filename.replace('.jpg', 'Z_value_line.jpg'))
        cv2.imwrite(output_image_filename, image)

# 將 mm_value 值儲存到 TXT 文件
mm_value_output_filename = os.path.join(output_txt_directory, "Z_coordinates_values.txt")
with open(mm_value_output_filename, 'w') as f:
    for mm_value in mm_value_list:
        f.write(f"{mm_value:.2f}\n")

print(f"All Z_value_line images of red regions saved at {output_image_directory}")
print(f"mm_value values saved at {mm_value_output_filename}")


All Z_value_line images of red regions saved at Z_value_line
mm_value values saved at Z_coordinates_output\Z_coordinates_values.txt


[Z_coordinates_values.txt](Z_coordinates_output\Z_coordinates_values.txt)

#### 3. 處理點雲 把 (x,y) 和 z 座標讀入 合併與旋轉 最終輸出

In [11]:
# 檔案路徑設定

input_coordinates_directory = "X_Y_coordinates_output"

output_points_cloud_directory = "Z_X_Y_points_cloud"
if not os.path.exists(output_points_cloud_directory):
    os.makedirs(output_points_cloud_directory)
#=========================================================

# 讀取 Z 座標值
z_values = []
z_values_file_path = os.path.join("Z_coordinates_output", "Z_coordinates_values.txt")
with open(z_values_file_path, 'r') as f:
    z_values = [float(line.strip()) for line in f.readlines()]

#==========================================================
all_coordinates = []  # 建立一個空列表，用於儲存所有的座標（包括 Z, X, Y）

# 遍歷 X_Y_coordinates_output 目錄中的所有文件，同時也遍歷 Z 座標值的列表
for filename, z_value in zip(os.listdir(input_coordinates_directory), z_values):
    # 確保只處理 .txt 文件
    if filename.endswith(".txt"):
        # 構建 .txt 文件的完整路徑
        input_file_path = os.path.join(input_coordinates_directory, filename)
        
        # 讀取 .txt 文件中的座標
        with open(input_file_path, 'r') as f:
            coordinates = f.readlines()  # 讀取所有行，每行代表一個座標
        
        # 將每個座標的 X 值、Y 值從座標中分離出來
        separated_coordinates = [coord.strip().split() for coord in coordinates]
        x_values = [float(coord[0]) for coord in separated_coordinates]
        y_values = [float(coord[1]) for coord in separated_coordinates]

        # 將每個座標的 Z 值加入到座標後面，形成 (Z, X, Y) 的格式
        # coordinates_with_z = [f"{coord.strip()} {z_value:.2f}\n" for coord in coordinates]
        coordinates_with_z = [f"{z_value:.2f} {coord.strip()}\n" for coord in coordinates]

        
        # 將含有 Z 值的座標加入到 all_coordinates 列表中
        all_coordinates.extend(coordinates_with_z)

# 將座標串在一起並儲存到一個 .xyz 檔案中
output_points_cloud_filename = os.path.join(output_points_cloud_directory, "Z_X_Y_points_cloud.xyz")
with open(output_points_cloud_filename, 'w') as f:
    for coord in all_coordinates:
        f.write(coord)


print(f"All Z, X, Y coordinates concatenated and saved at {output_points_cloud_filename}")

All Z, X, Y coordinates concatenated and saved at Z_X_Y_points_cloud\Z_X_Y_points_cloud.xyz


In [14]:
import numpy as np
import shutil

def rotate_xyz(input_filename, output_filename, angle_degrees):
    # 讀取原始的.xyz檔案
    with open(input_filename, 'r') as f:
        lines = f.readlines()

    # 提取座標部分，並進行旋轉
    rotated_lines = []
    for line in lines:
        if line.strip():  # 忽略空行
            coords = line.strip().split()
            x, y, z = float(coords[0]), float(coords[1]), float(coords[2])

            # 將座標點轉換為numpy array
            point = np.array([x, y, z])

            # 定義旋轉矩陣
            theta = np.radians(angle_degrees)
            rotation_matrix = np.array([[1, 0, 0],
                                         [0, np.cos(theta), -np.sin(theta)],
                                         [0, np.sin(theta), np.cos(theta)]])

            # 將座標點進行旋轉
            rotated_point = np.dot(rotation_matrix, point)

            # 將旋轉後的座標點添加到列表中
            rotated_lines.append(" ".join(str(coord) for coord in rotated_point) + "\n")

    # 將旋轉後的座標保存到新的.xyz檔案中
    with open(output_filename, 'w') as f:
        f.writelines(rotated_lines)

    print(f"Rotated XYZ coordinates saved to {output_filename}")
    # 複製旋轉後的檔案到程式碼同個目錄下
    shutil.copy(output_filename, os.getcwd())
# 測試程式
input_coordinates_directory = "Z_X_Y_points_cloud"
filename = "Z_X_Y_points_cloud.xyz"  # 原始的.xyz檔案名稱
input_file_path = os.path.join(input_coordinates_directory, filename)

output_directory = "Z_X_Y_points_cloud"

# 新的.xyz檔案名稱，保存旋轉後的座標
output_filename = os.path.join(output_directory, filename.replace('.xyz', '_rotated.xyz'))

angle_degrees = 180  # 旋轉角度（度）
rotate_xyz(input_file_path, output_filename, angle_degrees)


Rotated XYZ coordinates saved to Z_X_Y_points_cloud\Z_X_Y_points_cloud_rotated.xyz


### Result : Z_X_Y_points_cloud\Z_X_Y_points_cloud_rotated.xyz

[Z_X_Y_points_cloud_rotated.xyz](./Z_X_Y_points_cloud/Z_X_Y_points_cloud_rotated.xyz)